In [1]:
# 导入pandas与numpy工具包。
import pandas as pd
import numpy as np

# 创建特征列表。
column_names = [
    'Sample code number', 'Clump Thickness', 'Uniformity of Cell Size',
    'Uniformity of Cell Shape', 'Marginal Adhesion',
    'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
    'Normal Nucleoli', 'Mitoses', 'Class'
]

# 使用pandas.read_csv函数从互联网读取指定数据。
data = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data',
    names=column_names)

# 将?替换为标准缺失值表示。
data = data.replace(to_replace='?', value=np.nan)
# 丢弃带有缺失值的数据（只要有一个维度有缺失）。
data = data.dropna(how='any')

# 输出data的数据量和维度。
print(data.shape)

data.head()

(683, 11)


,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


数据集的简介见：[breast-cancer-wisconsin](https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names)

我们得知该数据集共有 $699$ 个样本，$11$ 个特征，其中一列特征表示检索的 id, $9$ 列与肿瘤相关的医学特征以及一列表示肿瘤类型的数值。`2` 表示 benign (良性), `4` 表示 malignant (恶性). 该数据集中有 $16$ 个缺失值，并使用 `?` 标出。

## 划分数据集

In [2]:
# 使用sklearn.cross_valiation里的train_test_split模块用于分割数据。
from sklearn.model_selection import train_test_split

# 随机采样25%的数据用于测试，剩下的75%用于构建训练集合。
X_train, X_test, y_train, y_test = train_test_split(
    data[column_names[1:10]],
    data[column_names[10]],
    test_size=0.25,
    random_state=33)

In [3]:
# 转换数据类型
X_train = X_train.astype(float)
X_test = X_test.astype(float)

In [4]:
# 查验训练样本的数量和类别分布。
y_train.value_counts()

2    344
4    168
Name: Class, dtype: int64

In [5]:
# 查验测试样本的数量和类别分布。
y_test.value_counts()

2    100
4     71
Name: Class, dtype: int64

## 数据集的标准化

In [6]:
# 从sklearn.preprocessing里导入StandardScaler。
from sklearn.preprocessing import StandardScaler
# 从sklearn.linear_model里导入LogisticRegression与SGDClassifier。
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

# 标准化数据，保证每个维度的特征数据方差为1，均值为0。使得预测结果不会被某些维度过大的特征值而主导。
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

# 使用 sklearn 进行  Logistics Regression 学习

In [20]:
# 初始化LogisticRegression与SGDClassifier。
lr = LogisticRegression(solver='lbfgs')
sgdc = SGDClassifier(max_iter=10)

In [21]:
%%timeit
# 调用LogisticRegression中的fit函数/模块用来训练模型参数。
lr.fit(X_train, y_train)

2.89 ms ± 141 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit
# 调用SGDClassifier中的fit函数/模块用来训练模型参数。
sgdc.fit(X_train, y_train)

652 µs ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
# 使用训练好的模型lr对X_test进行预测，结果储存在变量lr_y_predict中。
lr_y_predict = lr.predict(X_test)

In [24]:
# 使用训练好的模型sgdc对X_test进行预测，结果储存在变量sgdc_y_predict中。
sgdc_y_predict = sgdc.predict(X_test)

In [25]:
# 从sklearn.metrics里导入classification_report模块。
from sklearn.metrics import classification_report

# 使用逻辑斯蒂回归模型自带的评分函数score获得模型在测试集上的准确性结果。
print('Accuracy of LR Classifier:', lr.score(X_test, y_test))
# 利用classification_report模块获得LogisticRegression其他三个指标的结果。
print(
    classification_report(
        y_test, lr_y_predict, target_names=['Benign', 'Malignant']))

Accuracy of LR Classifier: 0.9883040935672515
              precision    recall  f1-score   support

      Benign       0.99      0.99      0.99       100
   Malignant       0.99      0.99      0.99        71

   micro avg       0.99      0.99      0.99       171
   macro avg       0.99      0.99      0.99       171
weighted avg       0.99      0.99      0.99       171



In [27]:
# 使用随机梯度下降模型自带的评分函数score获得模型在测试集上的准确性结果。
print('Accuarcy of SGD Classifier:', sgdc.score(X_test, y_test))
# 利用classification_report模块获得SGDClassifier其他三个指标的结果。
print(
    classification_report(
        y_test, sgdc_y_predict, target_names=['Benign', 'Malignant']))

Accuarcy of SGD Classifier: 0.9824561403508771
              precision    recall  f1-score   support

      Benign       1.00      0.97      0.98       100
   Malignant       0.96      1.00      0.98        71

   micro avg       0.98      0.98      0.98       171
   macro avg       0.98      0.98      0.98       171
weighted avg       0.98      0.98      0.98       171



一般说来，`SGDClassifier` 训练速度快于 `LogisticRegression`，适用于 $10$ 万量级的数据；而 `LogisticRegression` 的参数是采用精确解析的方式,模型的精度会相对高一点, 但是对于 $10$ 万量级的数据内存开销很大，有点不切实际。

 ---------------------

# 使用 MXNet 实现 Logistics Regression

使用 SGD 进行优化

In [35]:
import random
import mxnet as mx
from mxnet import autograd, nd


def data_iter(batch_size, features, labels, trainable=True):
    '''
    数据迭代器
    '''
    num_examples = len(features)
    indices = np.arange(num_examples)
    if trainable:
        random.shuffle(indices)  # 样本的读取顺序是随机的。
    for i in range(0, num_examples, batch_size):
        j = np.array(indices[i: min(i + batch_size, num_examples)])
        # take 函数根据索引返回对应元素。
        yield features.take(j, axis=0), labels.take(j, axis=0)


def LR(X, w, b):
    '''
    Logistics Regression
    '''
    z = nd.dot(X, w) + b
    exp = 1 / (1 + z.exp())
    return exp


def squared_loss(y_hat, y):
    '''
    损失函数
    '''
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2


def sgd(params, lr, batch_size):
    '''
    优化算法
    '''
    for param in params:
        param[:] -= lr * param.grad / batch_size

In [36]:
num_features = X_train.shape[1]
w = nd.random.normal(shape=(num_features, 1))+0.001
b = nd.zeros(1)
w.attach_grad()
b.attach_grad()

In [37]:
lr = 0.1
num_epochs = 100
batch_size = 64

for epoch in range(num_epochs):  # 训练模型一共需要 num_epochs 个迭代周期。
    # 在一个迭代周期中，使用训练数据集中所有样本一次（假设样本数能够被批量大小整除）。
    # X 和 y 分别是小批量样本的特征和标签。
    for X, y in data_iter(32, X_train, y_train):
        X = nd.array(X)/255
        y = nd.array(y)
        with autograd.record():
            y_hat = LR(X, w, b)
            L = squared_loss(y_hat, y)
        L.backward()
        sgd([w, b], lr, batch_size)  # 使用小批量随机梯度下降迭代模型参数。
    train_l = squared_loss(LR(X, w, b), y)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))

epoch 1, loss 2.928271
epoch 2, loss 2.019408
epoch 3, loss 2.892815
epoch 4, loss 2.237970
epoch 5, loss 1.769795
epoch 6, loss 2.129934
epoch 7, loss 1.562507
epoch 8, loss 1.538613
epoch 9, loss 1.913836
epoch 10, loss 2.418076
epoch 11, loss 2.273044
epoch 12, loss 2.389298
epoch 13, loss 1.602880
epoch 14, loss 2.368069
epoch 15, loss 2.102364
epoch 16, loss 1.581749
epoch 17, loss 2.730835
epoch 18, loss 1.956262
epoch 19, loss 1.312663
epoch 20, loss 1.436876
epoch 21, loss 1.688527
epoch 22, loss 1.813171
epoch 23, loss 1.555527
epoch 24, loss 1.425814
epoch 25, loss 2.059360
epoch 26, loss 1.929702
epoch 27, loss 1.800970
epoch 28, loss 2.179071
epoch 29, loss 2.050675
epoch 30, loss 2.048612
epoch 31, loss 1.160229
epoch 32, loss 1.285836
epoch 33, loss 2.423917
epoch 34, loss 1.789774
epoch 35, loss 2.294211
epoch 36, loss 1.787175
epoch 37, loss 1.786389
epoch 38, loss 1.784949
epoch 39, loss 2.289278
epoch 40, loss 1.530488
epoch 41, loss 1.656279
epoch 42, loss 1.781740
e

In [38]:
LR_y_predict = LR(nd.array(X_test), w, b).asnumpy()
LR_y_predict = np.where(LR_y_predict > .5, 2, 4)

In [39]:
# 从sklearn.metrics里导入classification_report模块。
from sklearn.metrics import classification_report

print(
    classification_report(
        y_test<3, LR_y_predict<3, target_names=['Benign', 'Malignant']))

              precision    recall  f1-score   support

      Benign       1.00      0.34      0.51        71
   Malignant       0.68      1.00      0.81       100

   micro avg       0.73      0.73      0.73       171
   macro avg       0.84      0.67      0.66       171
weighted avg       0.81      0.73      0.68       171



In [40]:
y_hat = nd.array([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = nd.array([0, 1])
nd.pick(y_hat, y)


[0.1 0.2]
<NDArray 2 @cpu(0)>

In [41]:
y_hat


[[0.1 0.3 0.6]
 [0.3 0.2 0.5]]
<NDArray 2x3 @cpu(0)>